# 2. Pré-processamento e Limpeza (Avançado) - Dataset 2

Este notebook realiza a limpeza e preparação dos dados para modelagem, incluindo tratamento de emojis, gírias e negação.

## Objetivos
- Limpeza de texto (preservando pontuação relevante).
- Tratamento de Emojis (demojize).
- Normalização de Gírias.
- Marcação de Negação.
- Tokenização e remoção de stopwords.
- Criação do alvo `sentiment` (binário).
- Exportação do dataset processado para `datasets/processed/reviews_dataset2_advanced.csv`.

In [6]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji
import os

# Configuração inicial
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('portuguese'))
# Remover palavras de negação das stopwords para não atrapalhar a marcação de negação
negation_terms = {'não', 'nao', 'nem', 'nunca', 'jamais', 'nada', 'ninguém', 'ninguem'}
stop_words = stop_words - negation_terms

## Carregamento dos Dados

In [7]:
file_path = '../datasets/reviews_mercadolivre_com_br_2.json'
df = pd.read_json(file_path)
df.head()

,date,rating,content,product_url
0,01 fev. 2025,5,Muito bom gostei.,https://produto.mercadolivre.com.br/MLB-357687...
1,22 jan. 2025,5,"Perfeito. Eu simplesmente amei , ele e maravil...",https://produto.mercadolivre.com.br/MLB-357687...
2,12 jan. 2025,5,Muito cheiroso e a consistência e maravilhosa.,https://produto.mercadolivre.com.br/MLB-357687...
3,29 nov. 2024,4,,https://produto.mercadolivre.com.br/MLB-404730...
4,12 fev. 2025,5,Produto maravilhoso.,https://produto.mercadolivre.com.br/MLB-404730...


## Dicionário de Gírias

In [8]:
slang_map = {
    'vc': 'você',
    'vcs': 'vocês',
    'tb': 'também',
    'tbm': 'também',
    'pq': 'porque',
    'q': 'que',
    'k': '', 'kk': '', 'kkk': '', 'kkkk': '', 'rs': '',
    'mto': 'muito',
    'mt': 'muito',
    'ta': 'está',
    'eh': 'é',
    'hj': 'hoje',
    'td': 'tudo',
    'blz': 'beleza',
    'flw': 'falou',
    'abs': 'abraços',
    'tmj': 'tamo junto',
    'n': 'não',
    's': 'sim',
    'obg': 'obrigado',
    'top': 'ótimo',
    'show': 'ótimo'
}

def normalize_slang(text, slang_map):
    words = text.split()
    normalized_words = [slang_map.get(word, word) for word in words]
    return " ".join(normalized_words)

## Limpeza e Tratamento Avançado

In [ ]:
def clean_text_advanced(text):
    if not isinstance(text, str):
        return ""
    
    # Tratar Emojis (Demojize): 👍 -> :thumbs_up:
    # Usamos language='pt' se disponível na versão, senão padrão en
    text = emoji.demojize(text, language='pt') 
    
    # Converter para minúsculas
    text = text.lower()
    
    # Substituir hifens por espaço
    text = text.replace('-', ' ')
    
    # Normalizar Gírias
    text = normalize_slang(text, slang_map)
    
    # Remover caracteres especiais MAS manter pontuação relevante (! ?)
    # Regex: Mantém letras, números, espaços, acentos, !, ?, :, _ (para emojis)
    text = re.sub(r'[^a-zA-Z0-9\u00C0-\u00FF\s!\?:_]', '', text)
    
    # Espaçar pontuação para ser tokenizada separadamente "Bom!" -> "Bom !"
    text = re.sub(r'([!\?])', r' \1 ', text)
    
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def mark_negation_func(text_tokens):
    """
    Marca negação em tokens até a próxima pontuação.
    Ex: ['não', 'gostei', 'do', 'produto', '.'] -> ['não', 'gostei_NEG', 'do_NEG', 'produto_NEG', '.']
    """
    negations = negation_terms
    punctuation = {'!', '?', '.', ',', ':', ';', '...'}
    
    result = []
    neg_state = False
    
    for word in text_tokens:
        # Se for pontuação, reseta negação
        if word in punctuation:
            neg_state = False
            result.append(word)
            continue
            
        # Se palavra atual é termo de negação, ativa estado (e mantém a palavra original)
        if word in negations:
            neg_state = True
            result.append(word)
            continue
            
        # Se estado ativo, adiciona sufixo _NEG
        if neg_state:
            result.append(word + '_NEG')
        else:
            result.append(word)
            
    return result

def process_text_pipeline(text):
    # Limpeza básica
    cleaned = clean_text_advanced(text)
    
    # Tokenização
    tokens = word_tokenize(cleaned, language='portuguese')
    
    # Remover stopwords (lembrando que tiramos as de negação da lista)
    tokens = [t for t in tokens if t not in stop_words]
    
    # Marcação de Negação
    tokens = mark_negation_func(tokens)
    
    # Filtrar tokens muito curtos (opcional, cuidado com emojis :a:)
    # Vamos manter tokens > 1 ou emojis
    final_tokens = [t for t in tokens if len(t) > 1 or t in ['!', '?']]
    
    return " ".join(final_tokens)

# Aplicação
df['processed_text'] = df['content'].apply(process_text_pipeline)
df[['content', 'processed_text']].head(10)

,content,processed_text
0,Muito bom gostei.,bom gostei
1,"Perfeito. Eu simplesmente amei , ele e maravil...",perfeito simplesmente amei maravilhoso super h...
2,Muito cheiroso e a consistência e maravilhosa.,cheiroso consistência maravilhosa
3,,
4,Produto maravilhoso.,produto maravilhoso
5,Muito bom. Veio certinho. Agora é usar pra sab...,bom veio certinho agora usar pra saber resulta...
6,Gosto muito.,gosto
7,Amei!!! para meu cabelo foi ótimo.,amei ! ! ! cabelo ótimo
8,Muito bom bem consistente custo benefício ótimo.,bom bem consistente custo benefício ótimo
9,"Hidrata muito,amei.",hidrata muitoamei


## Criação do Target e Exportação

In [10]:
df['sentiment'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)
df = df.rename(columns={'content': 'review_text'})

# Salvar com novo nome para manter versão anterior
output_dir = '../datasets/processed'
output_file = os.path.join(output_dir, 'reviews_dataset2_advanced.csv')
os.makedirs(output_dir, exist_ok=True)

df[['review_text', 'rating', 'sentiment', 'processed_text']].to_csv(output_file, index=False)

print(f"Dados avançados salvos em: {output_file}")

Dados avançados salvos em: ../datasets/processed/reviews_dataset2_advanced.csv
